The circuit below adds two numbers using QFT for 6 bits each and an output that fits into 6 bits. The steps are below. 

- Load the values using basis embeddeding into the 12 qubits. 
- Apply QFT into the target qubits to shift them into fourier basis. 
- Load value of A onto target qubits. Use the formula $ \frac{2*A*pi}{2^j} $ where j = 0,1,2,3,4,5,6
- Load value of B onto target qubits. Use the formula $ \frac{2*B*pi}{2^j} $ where j = 0,1,2,3,4,5,6
- Apply IQFT to get the value. 
- Use simulator to get the values.

In [253]:
from qiskit import *
import math 
from qiskit.visualization import plot_state_city, plot_bloch_multivector, plot_histogram
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
import numpy as np
from random import randrange

def add_numbers(A,B):
    tbits_a = 6
    tbits_b = 6
    target_bits = 6

    total = tbits_a+tbits_b+target_bits
    QR = QuantumRegister(total)
    CR = ClassicalRegister(target_bits)
    QC = QuantumCircuit(QR, CR)

    bits_a = format(A, '#010b')[::-1]
    bits_b = format(B, '#010b')[::-1]

    for bit,idx in zip(bits_a, range(len(bits_a))):
        if bit == '1':
            QC.x(idx)

    for bit,idx in zip(bits_b, range(len(bits_b))):
        if bit == '1':
            QC.x(4+idx)

    QC.append(QFT(target_bits, 0, False), list(range(tbits_a+tbits_b, total)))
    QC.barrier()

    for i in range(target_bits):
        QC.rz(A*math.pi/(2**i), i+tbits_a+tbits_b)

    for i in range(target_bits):
        QC.rz(B*math.pi/(2**i), i+tbits_a+tbits_b)    

    QC.barrier()

    QC.append(QFT(target_bits, 0, False, True), list(range(tbits_a+tbits_b, total)))
    
    for i in range(target_bits):
        QC.measure(i+tbits_a+tbits_b,i)

    backend = BasicAer.get_backend('qasm_simulator') # the device to run on
    result = backend.run(transpile(QC, backend), shots=1).result()
    counts  = result.get_counts(QC)
    value = list(counts.keys())[0]
    return (QC, (int(value, 2)))

for i in range(10):
    x, y = randrange(0,25), randrange(0,25)
    circ, ans = add_numbers(x,y)
    print (x,y, ans)
    circ.draw()
    


24 19 43
16 1 17
15 22 37
20 6 26
12 15 27
20 5 25
19 11 30
19 3 22
1 21 22
16 16 32
